# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Multiple sclerosis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = False

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [20]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [16]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [17]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [18]:
gender_col = 'gender'

In [19]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [23]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Thyroid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Multiple-sclerosis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE171950',
 'GSE193442',
 'GSE189788',
 '.DS_Store',
 'GSE215450',
 'GSE153459',
 'GSE224457',
 'GSE190847']

Repeat the below steps for all the accession numbers

In [39]:
cohort = accession_num = "GSE215450"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Multiple-sclerosis/GSE215450/GSE215450_family.soft.gz',
 '../../DATA/GEO/Multiple-sclerosis/GSE215450/GSE215450_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [40]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Circulating microRNAs in relapsing MS patients treated with dimethyl fumarate in the phase 4 TREMEND trial"
!Series_summary	"Dimethyl fumarate (DMF) is an oral drug approved for relapsing multiple sclerosis (MS) that leads to reduction of neurofilament light (NFL).  This may be related to dynamics and persistence of microRNA signatures in the peripheral blood of treatment-naïve MS patients before and after dimethyl fumarate (DMF) at different time points. 210 blood samples were collected from 51 treatment-naïve patients at baseline (BL) and after 1-3, 4-7, 9-15 and 21-27 months of DMF and from 22 controls from the phase IV TREMEND trial. Using microarray, 1,085 miRNAs were two-folds above the background and compared versus NFL. Altered miRNA profiles peaked after 4-7 months. MiR-16-5p and miR-4306, involved in the NF-kB-pathway, were upregulated in low NFL samples, while miR-940 and miR-4665-3p were upregulated in high NFL samples. NFL and miRNA correlations were stronge

In [41]:
clinical_data.head()

,!Sample_geo_accession,GSM6668434,GSM6668435,GSM6668436,GSM6668437,GSM6668438,GSM6668439,GSM6668440,GSM6668441,GSM6668442,...,GSM6668635,GSM6668636,GSM6668637,GSM6668638,GSM6668639,GSM6668640,GSM6668641,GSM6668642,GSM6668643,GSM6668644
0,!Sample_characteristics_ch1,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,...,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood,tissue: whole blood
1,!Sample_characteristics_ch1,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,timepoint: Control,...,timepoint: 1-year group,timepoint: Long term group,timepoint: Baseline group,timepoint: Early group,timepoint: Early group,timepoint: 1-year group,timepoint: Baseline group,timepoint: Intermediate group,timepoint: 1-year group,timepoint: 1-year group
2,!Sample_characteristics_ch1,age: 51,age: 29,age: 35,age: 31,age: 43,age: 30,age: 22,age: 30,age: 23,...,age: 22,age: 22,age: 40,age: 40,age: 40,age: 40,age: 27,age: 27,age: 27,age: 27
3,!Sample_characteristics_ch1,Sex: female,Sex: female,Sex: female,Sex: male,Sex: female,Sex: male,Sex: female,Sex: female,Sex: female,...,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female,Sex: female


In [42]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: whole blood'],
 1: ['timepoint: Control',
  'timepoint: Baseline group',
  'timepoint: Early group',
  'timepoint: 1-year group',
  'timepoint: Long term group',
  'timepoint: Intermediate group'],
 2: ['age: 51',
  'age: 29',
  'age: 35',
  'age: 31',
  'age: 43',
  'age: 30',
  'age: 22',
  'age: 23',
  'age: 40',
  'age: 27',
  'age: 44',
  'age: 49',
  'age: 41',
  'age: 46',
  'age: 48',
  'age: 19',
  'age: 20',
  'age: 24',
  'age: 26',
  'age: 25',
  'age: 33',
  'age: 47',
  'age: 37',
  'age: 32',
  'age: 28',
  'age: 34',
  'age: 18',
  'age: 36',
  'age: 42',
  'age: 38'],
 3: ['Sex: female', 'Sex: male']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [43]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Multiple sclerosis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Multiple sclerosis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. Th

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [44]:
age_row = gender_row = None
convert_age = convert_gender = None

In [45]:
is_gene_availabe = True
trait_row = 1
age_row = 2
gender_row = 3

trait_type = 'binary'

In [46]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [49]:
def convert_trait(value):
    """
    Converts a given value to a binary representation for the presence of Multiple Sclerosis.
    Returns 1 if MS is present, 0 if healthy control, and None for unknown values.
    """
    if 'Baseline' in value or 'Early' in value:
        return 1
    elif 'Control' in value:
        return 0
    else:
        return None

def convert_age(value):
    """
    Converts a given value to a continuous data type for age.
    Returns the age as an integer, or None for unknown values.
    """
    try:
        return int(value.split(': ')[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    """
    Converts a given value to a binary representation for gender.
    Returns 1 for female, 0 for male, and None for unknown values.
    """
    if value == 'Sex: female':
        return 1
    elif value == 'Sex: male':
        return 0
    else:
        return None


In [50]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM6668434,GSM6668435,GSM6668436,GSM6668437,GSM6668438,GSM6668439,GSM6668440,GSM6668441,GSM6668442,GSM6668443,...,GSM6668635,GSM6668636,GSM6668637,GSM6668638,GSM6668639,GSM6668640,GSM6668641,GSM6668642,GSM6668643,GSM6668644
Multiple sclerosis,0,0,0,0,0,0,0,0,0,0,...,None,None,1,1,1,None,1,None,None,None
Age,51,29,35,31,43,30,22,30,23,40,...,22,22,40,40,40,40,27,27,27,27
Gender,1,1,1,0,1,0,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


### Genetic data preprocessing and final filtering

In [51]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM6668434,GSM6668435,GSM6668436,GSM6668437,GSM6668438,GSM6668439,GSM6668440,GSM6668441,GSM6668442,GSM6668443,...,GSM6668635,GSM6668636,GSM6668637,GSM6668638,GSM6668639,GSM6668640,GSM6668641,GSM6668642,GSM6668643,GSM6668644
ID,,,,,,,,,,,,,,,,,,,,,
hsa-let-7a-3p,3.569062,3.990944,3.958360,3.739480,3.558173,3.857273,4.090974,3.493339,3.692938,3.864426,...,3.860735,3.941018,3.502343,3.908902,3.505673,3.805248,3.716572,3.756652,3.889881,3.984206
hsa-let-7a-5p,4.298654,3.915365,4.786878,3.777195,4.203269,4.481954,4.956935,3.866321,5.224822,4.276339,...,4.658320,4.993832,4.597233,3.882999,5.173708,3.584311,4.613917,4.745288,4.213337,3.782079
hsa-let-7b-3p,5.262347,5.154800,4.992272,5.107225,5.146336,5.451357,5.298464,4.570695,4.784492,4.794610,...,5.063779,5.071583,4.893366,5.058334,5.033599,5.376854,5.183216,4.952687,5.130082,5.402780
hsa-let-7b-5p,5.240960,5.085624,5.320981,5.200781,4.999822,6.254998,5.516254,5.235488,5.640803,4.742894,...,5.589816,5.935900,5.736466,5.592717,5.908696,4.418627,6.020542,5.454368,6.064787,4.510776
hsa-let-7c-3p,4.174805,3.976034,3.427550,3.822488,3.850354,4.044914,3.899470,3.866420,3.671528,3.924554,...,4.037483,4.038049,3.623681,3.945570,3.852142,3.863140,3.773484,3.818517,3.981694,3.954567


In [52]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['hsa-let-7a-3p',
 'hsa-let-7a-5p',
 'hsa-let-7b-3p',
 'hsa-let-7b-5p',
 'hsa-let-7c-3p',
 'hsa-let-7c-5p',
 'hsa-let-7d-3p',
 'hsa-let-7d-5p',
 'hsa-let-7e-3p',
 'hsa-let-7e-5p',
 'hsa-let-7f-1-3p',
 'hsa-let-7f-2-3p',
 'hsa-let-7f-5p',
 'hsa-let-7g-3p',
 'hsa-let-7g-5p',
 'hsa-let-7i-3p',
 'hsa-let-7i-5p',
 'hsa-miR-1-3p',
 'hsa-miR-1-5p',
 'hsa-miR-100-3p']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [53]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['hsa-let-7a-3p', 'hsa-let-7a-5p', 'hsa-let-7b-3p', 'hsa-let-7b-5p', 'hsa-let-7c-3p', 'hsa-let-7c-5p', 'hsa-let-7d-3p', 'hsa-let-7d-5p', 'hsa-let-7e-3p', 'hsa-let-7e-5p', 'hsa-let-7f-1-3p', 'hsa-let-7f-2-3p', 'hsa-let-7f-5p', 'hsa-let-7g-3p', 'hsa-let-7g-5p', 'hsa-let-7i-3p', 'hsa-let-7i-5p', 'hsa-miR-1-3p', 'hsa-miR-1-5p', 'hsa-miR-100-3p']\n"

If not required, jump directly to the gene normalization step

In [54]:
requires_gene_mapping = True

In [55]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'miRNA_ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [56]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_name_key = key_name1
gene_name_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_name_key = key_name1\ngene_name_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'miRNA_ID': ['hsa-miR-3132', 'hsa-miR-449c-5p', 'hsa-miR-4481', 'hsa-miR-6788-5p', 'hsa-miR-514a-3p'], 'SPOT_ID': [nan, nan, nan, nan, nan]}\n"

In [58]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'miRNA_ID'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [59]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [60]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [61]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 119 samples.


In [62]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Multiple sclerosis', the least common label is '0.0' with 22 occurrences. This represents 18.49% of the dataset.
The distribution of the feature 'Multiple sclerosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 32.0
  75%: 40.0
Min: 18.0
Max: 51.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 26 occurrences. This represents 21.85% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [63]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [64]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
3,GSE215450,True,True,False,True,True,119.0,
0,GSE171950,True,True,False,False,False,30.0,


In [5]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE215450'

In [6]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
3,GSE215450,True,True,False,True,True,119.0,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Multiple sclerosis,Age,Gender,1.988332412,10.00244803,10.0026506,10.00987169,10.01043269,10.01244087,10.01740226,...,hsa-miR-95-5p,hsa-miR-9500,hsa-miR-96-3p,hsa-miR-96-5p,hsa-miR-98-3p,hsa-miR-98-5p,hsa-miR-99a-3p,hsa-miR-99a-5p,hsa-miR-99b-3p,hsa-miR-99b-5p
0,0.0,51.0,1.0,2.855409,10.017402,10.739442,10.017402,10.739442,10.017402,10.017402,...,3.327912,2.947059,2.982349,3.768332,4.969960,3.459125,3.504149,3.448990,3.203794,3.396792
1,0.0,29.0,1.0,2.606522,9.601527,9.940958,9.601527,9.940958,9.601527,9.601527,...,3.199212,3.094012,3.034922,3.535213,4.988086,3.499802,3.513327,3.637594,3.337921,3.446165
2,0.0,35.0,1.0,2.477578,9.849524,10.957172,9.849524,10.957172,9.849524,9.849524,...,3.346556,2.941218,2.928636,3.597025,4.677913,3.596995,3.440173,3.702877,3.136363,3.410131
3,0.0,31.0,0.0,2.694981,7.979865,11.753147,7.979865,11.753147,7.979865,7.979865,...,3.219165,3.162482,3.439317,3.759911,4.543207,3.229087,3.325727,3.634383,3.411122,3.518127
4,0.0,43.0,1.0,2.391146,9.543299,10.119571,9.543299,10.119571,9.543299,9.543299,...,3.454079,3.055734,3.174454,3.692791,5.031158,3.570276,3.405138,3.418141,3.120018,3.212788


In [8]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [9]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [10]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [11]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 81.74% ± 7.48%


In [12]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 56.52% ± 4.76%


In [13]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [14]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0578
Found 0 genes with non-zero coefficients associated with the trait 'Multiple sclerosis' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Empty DataFrame
Columns: [Variable, Coefficient, Absolute Coefficient]
Index: []


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [16]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0578
Found 0 significant genes associated with the trait 'Multiple sclerosis' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
